In [1]:
DB_HOST    = 'ec2-54-237-124-68.compute-1.amazonaws.com'
DB_USER    = 'tutwpjujzkfpym'
DB_PORT    = '5432'
DB_PASS    = '5e3ec8bc2661a3285c6324297a2dc2d589074407119c4550ebb1b69355c08dec'
DB_NAME    = 'dt34hsa1qkeo6'
DB_TABLE   = 'data_engine_recomender'

In [2]:
import datetime
import psycopg2

class DataBase:

    TABLE_NAME = DB_TABLE

    def __init__(self):
        self.conn = self.connect_postgres()
        self.cur = self.conn.cursor()

    def connect_postgres(self):
        # connect to PostgreSQL
        print ("\nconnecting to PostgreSQL")
        try:
            conn = psycopg2.connect(
                dbname=DB_NAME, user=DB_USER, password=DB_PASS,
                host=DB_HOST, port=DB_PORT
            )
        except Exception as err:
            print ("PostgreSQL Connect() ERROR:", err)
            conn = None
        # return the connection object
        return conn

    def create_table():
        self.cur.execute("""
            CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
                id SERIAL primary key, 
                video_title TEXT, 
                video_link TEXT UNIQUE, 
                thumbnail TEXT, 
                score FLOAT, 
                liked BOOLEAN DEFAULT FALSE,
                created_at TIMESTAMP DEFAULT NOW(),
                updated_at DATE);
        """)
        self.conn.commit()
        self.close()   

    def show_videos(self, qtd_videos = 20):
        """ Return list of videos for Front End """
        query = '''select id, video_title, video_link, thumbnail, score, liked from {TABLE_NAME}
                    order by score desc limit {qtd_videos}'''
        videos = self.cur.execute(query)
        result = videos.fetchall()
        self.close() 
        return result

    def get_by_link(self, video_link):
        """ Return one video by its link """
        query = '''SELECT id from {TABLE_NAME} where video_link = "{video_link}" '''
        video = self.cur.execute(query)
        self.close() 
        return video.fetchone()

    def save_recomendation(self, video_info):
        ''' check if video already exists '''
        video_exists = self.get_by_link(video_info['video_id'])

        if video_exists:
            return True

        video_title = video_info['title'].replace('"', "'"),
        video_link = video_info['video_id'],
        thumbnail = video_info['thumbnail'],
        score = video_info['score']

        query = '''INSERT INTO {TABLE_NAME} (video_title, video_link, thumbnail, score) values 
                                ("{video_title}", "{video_link}", "{thumbnail}", {score});
                '''
        new_video = self.cur.execute(query)
        self.close()
    
    def like_video(self, liked_value, video_id):
        """" Edit liked column """
        update_time = datetime.datetime.now().strftime("%Y %m %d %H:%M:%S")


        query = '''update {TABLE_NAME} set liked = {liked_value},
                        updated_at = '{update_time}' where id = {video_id}
                '''
        self.cur.execute(query)
        self.close()

    def close(self):
        self.cur.close()
        self.conn.close()

In [3]:
import pandas as pd
import re
import pickle
import os
import joblib as jb
from scipy.sparse import hstack, csr_matrix
from scipy import sparse
import numpy as np
import json

# mdl_rf = pickle.load( open( "random_forest_20200911", "rb" ) )
# mdl_lgbm = pickle.load( open( "lgbm_20200911", "rb" ) )
# title_vec = pickle.load( open( "title_vectorizer_20200911", "rb" ) )

ROOT_DIR = os.path.dirname(os.path.abspath(__file__))
#  '/' for POSIX and '\\' for Windows

ml_logistic = jb.load("logistic_reg_2021-12-23.pkl.z")
title_vec = jb.load("title_vectorizer_2021-12-23.pkl.z")

def clean_date(data):
    return pd.to_datetime(data['upload_date'], format="%Y-%m-%d")

def clean_views(data):
    raw_views_str = data['view_count']
    if raw_views_str is None:
        return 0
    return int(raw_views_str)

def compute_features(data):

    publish_date = clean_date(data)
    if publish_date is None:
        return None

    views = clean_views(data)
    title = data['title']

    features = dict()

    features['tempo_desde_pub'] = (pd.Timestamp.today() - publish_date) / np.timedelta64(1, 'D')
    features['views'] = views
    features['views_por_dia'] = features['views'] / features['tempo_desde_pub']
    del features['tempo_desde_pub']

    vectorized_title = title_vec.transform([title])

    num_features = sparse.csr_matrix(np.array([features['views'], features['views_por_dia']]))
    feature_array = sparse.hstack([num_features, vectorized_title])

    return feature_array


def compute_prediction(data):
    feature_array = compute_features(data)

    if feature_array is None:
        return 0

    # p_rf = mdl_rf.predict_proba(feature_array)[0][1]
    # p_lgbm = mdl_lgbm.predict_proba(feature_array)[0][1]
    # p = 0.2*p_rf + 0.8*p_lgbm

    prob = ml_logistic.predict_proba(feature_array)[0][1]
    log_data(data, feature_array, proba)

    return p

def log_data(data, feature_array, p):
    #print(data)
    video_id = data.get('og:video:url', '')
    data['prediction'] = p
    data['feature_array'] = feature_array.todense().tolist()
    #print(video_id, json.dumps(data))

ModuleNotFoundError: No module named 'scipy'

In [ ]:
import time
import os
import pandas as pd
from threading import Thread
from requests_html import HTMLSession

queries = ['data+engineering', 'data+pipeline', "airflow"]


# Em suma: busca novos dados, aplica o modelo
def update_db():
    try:
        database = DataBase()
        for query in queries:
            
            for page in range(1,4):
                print(query, page)
                search_page = download_search_page(query, page)
                print("=========================")
                print(" *** search_page ***")
                print(search_page)
                print("=========================")
                video_list = parse_search_page(search_page)
                print("=========================")
                print(" *** video_list ***")
                print(video_list)
                print("=========================")
                df_videos = pd.DataFrame(video_list)
                
                for video in df_videos['link'].unique():
                    if database.get_by_link(video):
                        print("Already registered in database: {}".format(video))
                        continue
                    video_json_data = parse_video_page(video)
                    if(not video_json_data):
                        continue
                    
                    p = compute_prediction(video_json_data)

                    video_id = video
                    data_front = {
                        "title": video_json_data['title'],
                        "score": float(p),
                        "video_id": video_id,
                        "thumbnail": video_json_data['thumbnail']
                       }
                    data_front['update_time'] = time.time_ns()
                    print(video_id, json.dumps(data_front))
                    database.save_recomendation(data_front)
    except Exception as identifier:
        # os.remove("novos_videos.json")
        print("================", identifier)
        raise Exception('Internal Server Error')
    return True

    """
O que vai fazer:
 + Para cada query: 
  - Web Scraping
  - Buaca os dadoa de de uma pagina por query e por fim  os video
    - Buca novos videos desde que nao ja esstjam instalados 
      - é testtado com um select
      - si é permidido, o video é convertido
    - Com os dados no formato JSON faz a previsao

"""

In [ ]:
# Copy Paste
import requests as rq
import bs4 as bs4
import re
import time
import youtube_dl 
import asyncio
from requests_html import HTMLSession, AsyncHTMLSession

def download_search_page(query, page):
    url = "https://www.youtube.com/results?search_query={query}&sp=CAI%253D&p={page}"
    urll = url.format(query=query, page=page)
    session = HTMLSession()
    # enviado requisição para o youtube
    response = session.get(urll)
    print("=========================")
    print(" *** response ***")
    print(response)
    print("=========================")
    # executando Java-script
    # t = Thread(target=render_html)
    # ORIGINAL :: response.html.render(sleep=1)
    return response.html.html


def download_video_page(link):
    url = "https://www.youtube.com{link}"
    urll = url.format(link=link)
    response = rq.get(urll)
    link_name = re.search("v=(.*)", link).group(1)
    return response.text


def parse_search_page(page_html):
     # Processando a página html
    parsed = bs4.BeautifulSoup(page_html, "html.parser")
    video_list = []
     # Buscando todas as tags "a" com o id "video-title"
    ancors = parsed.find_all('a', attrs={"id": "video-title"})
    for e in ancors:
        link = e['href']
        title = e['title']
        data = {"link": link, "title": title}
        video_list.append(data)
    return video_list

def parse_video_page(video_link):
    URL = "https://www.youtube.com{link}"
    ydl = youtube_dl.YoutubeDL({"ignoreerrors": True, 'verbose':False})
    try:
        r = ydl.extract_info(url=URL.format(link=video_link), download=False)
        year = r['upload_date'][:4]
        month = r['upload_date'][4:6]
        day = r['upload_date'][6:]
    except Exception:
        return False
    
    video_info = {
        'uploader': r['uploader'],
        'title': r['title'],
        'upload_date': f"{year}-{month}-{day}",
        'user': r['uploader_id'],
        'view_count': r['view_count'],
        'like_count': r['like_count'],
        'dislike_count': 0,
        'thumbnail': r['thumbnail'],
        'width': r['width'],
        'height': r['height'],
        'categories': '|'.join(r['categories']) if r['categories'] is not None else None,
        'tags': '|'.join(r['tags']) if r['tags'] is not None else None,
        'channel_url': r['channel_url'],
        'description': r['description']
    }

    return video_info


## Atualizar

In [4]:
# data+engineering, 1 a 4
def download_search_page(query, page):
    url = "https://www.youtube.com/results?search_query={query}&sp=CAI%253D&p={page}"
    urll = url.format(query=query, page=page)
    session = HTMLSession()
    # enviado requisição para o youtube
    response = session.get(urll)
    print("=========================")
    print(" *** response ***")
    print(response)
    print("=========================")
    # executando Java-script
    # t = Thread(target=render_html)
    # ORIGINAL :: response.html.render(sleep=1)
    return response.html.html

In [5]:
# OLd de 01
import youtube_dl

queries = ['data+engineering', 'data+pipeline', "airflow"]
url = "https://www.youtube.com/results?search_query={query}&sp=EgIIAg%253D%253D&p={page}"

ydl = youtube_dl.YoutubeDL({"ignoreerrors": True, 'verbose':False, "date": "2021-12-12"})

# Demora cerca de 50min (indepndende da maquina)
resultados = []
for query in queries:
    # yt-search-1100 : Busque por até 1100 Videos para consulta (busca no max 35pag = 500videos) 
    # 570 para data-enginering, 646 para data-pipeline
    r = ydl.extract_info("ytsearch1000:{}".format(query), download=False, extra_info={"date": "2021-12-12"})
    for entry in r['entries']:
        # adicionar a query usada para pegar aquele video
        if entry is not None:
            entry['query'] = query
    resultados += r['entries']
    
# Filtragem, pois pode aparecer algum 'None' entao, ele deve ser retirada da lsita de videos
resultados = [e for e in resultados if e is not None]
len(resultados)

KeyboardInterrupt: 

### Get Search Page

In [6]:
import datetime
import youtube_dl

def download_search_page(keyword, n_of_videos):
    video_links = []
    today = datetime.datetime.now().strftime("%Y-%m-%d")
    ydl = youtube_dl.YoutubeDL({"ignoreerrors": True, 'verbose':False, "date": today})
    # Faz a busca usando 
    result_search = ydl.extract_info("ytsearch{}:{}".format(n_of_videos, keyword),
                     download=False, extra_info={"date": today})
    # Altera cada elemnto do resultado
    for video_link in result_search['entries']:
         if video_link is not None:
            video_link['query'] = keyword
    video_links += result_search['entries']
    return video_links
    

In [7]:
r_donwload_search = download_search_page('data+engineering', 5)

[download] Downloading playlist: data+engineering
[youtube:search] query "data+engineering": Downloading page 1
[youtube:search] playlist data+engineering: Downloading 5 videos
[download] Downloading video 1 of 5
[youtube] qWru-b6m030: Downloading webpage
[download] Downloading video 2 of 5
[youtube] SpaFPPByOhM: Downloading webpage
[download] Downloading video 3 of 5
[youtube] nQu6s8FPhfA: Downloading webpage
[youtube] nQu6s8FPhfA: Downloading MPD manifest
[download] Downloading video 4 of 5
[youtube] vmYaAzbv9xk: Downloading webpage
[download] Downloading video 5 of 5
[youtube] AS2EyYK4x_Q: Downloading webpage
[download] Finished downloading playlist: data+engineering


In [8]:
def get_only_url(obj):
    return obj['id']

In [9]:
r_filter = list(map(get_only_url, r_donwload_search))
r_filter

['qWru-b6m030', 'SpaFPPByOhM', 'nQu6s8FPhfA', 'vmYaAzbv9xk', 'AS2EyYK4x_Q']

In [ ]:
def get_only_url(list_videos):
    return list(map(lambda el: el['id'], r_donwload_search))

In [ ]:
get_only_url(r_donwload_search)

### Get Video Page

In [ ]:
def download_video_page(video_link):
    URL = "https://www.youtube.com/watch?v={link}"
    ydl = youtube_dl.YoutubeDL({"ignoreerrors": True, 'verbose':False})
    try:
        r = ydl.extract_info(url=URL.format(link=video_link), download=False)
        year = r['upload_date'][:4]
        month = r['upload_date'][4:6]
        day = r['upload_date'][6:]
    except Exception:
        return False
    
    video_info = {
        'link': video_link,
        'uploader': r['uploader'],
        'title': r['title'],
        'upload_date': f"{year}-{month}-{day}",
        'user': r['uploader_id'],
        'view_count': r['view_count'],
        'like_count': r['like_count'],
        'dislike_count': 0,
        'thumbnail': r['thumbnail'],
        'width': r['width'],
        'height': r['height'],
        'categories': '|'.join(r['categories']) if r['categories'] is not None else None,
        'tags': '|'.join(r['tags']) if r['tags'] is not None else None,
        'channel_url': r['channel_url'],
        'description': r['description']
    }

    return video_info

In [ ]:
for el in r_filter:
    print(download_video_page(el))

In [ ]:
video_list = list(map(download_video_page, get_only_url(r_donwload_search)))

In [ ]:
video_list

In [ ]:
df_videos = pd.DataFrame(video_list)
df_videos

In [ ]:
df_videos.columns